In [ ]:
import json
import pandas as pd
import re

In [ ]:
!wget -O /content/corpus1.tsv https://raw.githubusercontent.com/akutuzov/rushifteval_public/main/raw_annotations/rushifteval1_test_raw_annotations.tsv
!wget -O /content/corpus2.tsv https://raw.githubusercontent.com/akutuzov/rushifteval_public/main/raw_annotations/rushifteval2_test_raw_annotations.tsv
!wget -O /content/corpus3.tsv https://raw.githubusercontent.com/akutuzov/rushifteval_public/main/raw_annotations/rushifteval3_test_raw_annotations.tsv

--2024-03-24 20:01:05--  https://raw.githubusercontent.com/akutuzov/rushifteval_public/main/raw_annotations/rushifteval1_test_raw_annotations.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2136977 (2.0M) [text/plain]
Saving to: ‘/content/corpus1.tsv’

/content/corpus1.ts 100%[===================>]   2.04M  --.-KB/s    in 0.06s   

2024-03-24 20:01:05 (31.7 MB/s) - ‘/content/corpus1.tsv’ saved [2136977/2136977]

--2024-03-24 20:01:05--  https://raw.githubusercontent.com/akutuzov/rushifteval_public/main/raw_annotations/rushifteval2_test_raw_annotations.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.

In [ ]:
! pip install natasha

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 410.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 51.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=59b2bb2ada7d5d08c66eee8a0fc5a28586ce05ccc65712c9d82b957bba9de974
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26096 sha256=582329ac7c5a34fe6aaaf58b2cee478928bc5f0966e48e88e0eb5bec530c0dae
  Stored in directory: /root/.cache/pip/wheels/fa/80/8c/43488a924a046b733b64de3fac99252674c892a4c3801c0a61
Successfully built docopt intervaltree


In [ ]:
from natasha import (
    Segmenter,
    MorphVocab,

    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,

    PER,
    NamesExtractor,

    Doc
)


segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

In [ ]:
def lemmatizator(corpus):
  result=''
  doc=Doc(corpus)
  doc.segment(segmenter)
  doc.tag_morph(morph_tagger)
  for token in doc.tokens:
     token.lemmatize(morph_vocab)
  for token in doc.tokens:
    if token.pos!='PUNCT':
      result=result+' '+token.lemma
    else:
      result=result+token.lemma
  return(result[1:])

In [ ]:
def del_punct(s):
  new_s = re.sub("<b><i>|</i></b>", "", s)
  new_s = re.sub(r'[^\w\s]', '', new_s)
  return new_s

In [ ]:
def create_corp(file_path, corp={}, sent='sent1'):
  df = pd.read_csv(file_path, delimiter='\t')
  words = list(set(df['word']))
  for w in words:
    if w not in corp.keys():
      corp[w] = []
    sents = list(df[df['word'] == w][sent])
    for s in sents:
      corp[w].append(s)

  return corp

In [ ]:
def clean(corp):
  for w in corp.keys():
    for s in range(len(corp[w])):
      print()
      corp[w][s] = lemmatizator(del_punct(corp[w][s]))

  for w in corp.keys():
    corp[w] = list(set(corp[w]))

  return corp

In [ ]:
c1 = create_corp('/content/corpus1.tsv', corp={})

In [ ]:
c1 = create_corp('/content/corpus3.tsv', corp=c1)

In [ ]:
c1 = clean(c1)

Выходные данные были обрезаны до нескольких последних строк (5000).






































































































































































































































































































































































































































































































































































































































































































































































































































































































































































In [ ]:
c3 = create_corp('/content/corpus2.tsv', corp={}, sent='sent2')

In [ ]:
c3 = create_corp('/content/corpus3.tsv', corp=c3, sent='sent2')

In [ ]:
c3 = clean(c3)

Выходные данные были обрезаны до нескольких последних строк (5000).






































































































































































































































































































































































































































































































































































































































































































































































































































































































































































In [ ]:
c2 = create_corp('/content/corpus1.tsv', corp={}, sent='sent2')

In [ ]:
c2 = create_corp('/content/corpus2.tsv', corp=c2)

In [ ]:
c2 = clean(c2)

Выходные данные были обрезаны до нескольких последних строк (5000).






































































































































































































































































































































































































































































































































































































































































































































































































































































































































































In [ ]:
with open('corpus19.json', 'w', encoding='utf-8') as file:
    json.dump(c1, file)

In [ ]:
with open('corpus20.json', 'w', encoding='utf-8') as file:
    json.dump(c2, file)

In [ ]:
with open('corpus21.json', 'w', encoding='utf-8') as file:
    json.dump(c3, file)

In [ ]:
with open('corpus21.json') as f:
    aa = json.load(f)